In [ ]:
import pandas as pd
import pickle

from dataset import NewsDataset

from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient

from collections import defaultdict
import string

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
from transformers import DebertaConfig

from IPython.display import display, HTML

from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

2025-08-08 00:25:51.746131: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754612752.121566      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754612752.225421      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login

login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [7]:
checkpoint = "ConditionalNLI/Exp5_deberta-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/985 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

In [ ]:
config = DebertaConfig()
batch_size = 2
criterion = nn.CrossEntropyLoss()
config.max_position_embeddings

512

## Running on Dataset

In [9]:
from datasets import load_dataset, concatenate_datasets
import json
import ast

In [10]:
test_df4 = pd.read_excel('/kaggle/input/confer-extension/Part4B_Dataset/4B_Type4.xlsx')
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase,type
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,his wind tunnel,related
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,his content management application,related
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,related
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,his wetsuit,related
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,his pastry bag,related
...,...,...,...,...,...,...
292,"If Christina is a makeup artist, John's friend...",Christina has a makeup sponge.,N,possessive,John's friend,unrelated
293,"If Rahim is a clergyman, John's friend will ca...",Rahim has a holy book.,N,possessive,John's friend,unrelated
294,"If James is a police officer, John's friend wi...",James has a gun.,N,possessive,John's friend,unrelated
295,"If Maya is a piano teachis, John's friend will...",Maya has a metronome.,N,possessive,John's friend,unrelated


In [11]:
test_df5a = pd.read_excel('/kaggle/input/confer-extension/Part4B_Dataset/4B_Type5A.xlsx')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase,type
0,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,again,watch a movie by Nolan again,related
1,"If Emily passes her driving test, she'll never...",Emily has read a book by Dan Brown before.,E,again,read a book by Dan Brown again,related
2,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,again,eat a traditional Swedish dish again,related
3,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,again,sing a pop song again,related
4,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,again,listen to a pop song again,related
...,...,...,...,...,...,...
1595,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,N,again,obtain a job position,no again
1596,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,N,again,convey a confidential message,no again
1597,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,N,again,manage to keep a secret,no again
1598,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,N,again,cherish a moment in peace,no again


In [12]:
test_df5p = pd.read_excel('/kaggle/input/confer-extension/Part4B_Dataset/4B_Type5P.xlsx')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase,type
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,related
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,related
2,"If Scarlett finishes her work early, her husba...",Scarlett has a husband.,E,possessive,her husband,related
3,"If Chloe finishes his work early, his colleagu...",Chloe has colleagues.,E,possessive,his colleagues,related
4,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,possessive,her parents,related
...,...,...,...,...,...,...
1195,"If Dorothy car breaks down, Tina's friend will...",Dorothy has a brother.,N,possessive,Tina's friend,unrelated
1196,"If Susan finishes his work early, Tina's frien...",Susan has a colleague.,N,possessive,Tina's friend,unrelated
1197,"If Margaret finishes work early, Tina's friend...",Margaret has a wife.,N,possessive,Tina's friend,unrelated
1198,"If Betty misses the bus, Tina's friend will dr...",Betty has a brother.,N,possessive,Tina's friend,unrelated


In [13]:
test_df = pd.concat([test_df4,test_df5a,test_df5p])
test_df

,premise,hypothesis,gold_label,trigger,key_phrase,type
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,his wind tunnel,related
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,his content management application,related
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,related
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,his wetsuit,related
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,his pastry bag,related
...,...,...,...,...,...,...
1195,"If Dorothy car breaks down, Tina's friend will...",Dorothy has a brother.,N,possessive,Tina's friend,unrelated
1196,"If Susan finishes his work early, Tina's frien...",Susan has a colleague.,N,possessive,Tina's friend,unrelated
1197,"If Margaret finishes work early, Tina's friend...",Margaret has a wife.,N,possessive,Tina's friend,unrelated
1198,"If Betty misses the bus, Tina's friend will dr...",Betty has a brother.,N,possessive,Tina's friend,unrelated


In [14]:
test_df.to_csv('part4b_all.csv')

In [15]:
sample_test_df = test_df4.iloc[:8].reset_index()
sample_test_df

,index,premise,hypothesis,gold_label,trigger,key_phrase,type
0,0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,his wind tunnel,related
1,1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,his content management application,related
2,2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,related
3,3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,his wetsuit,related
4,4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,his pastry bag,related
5,5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,his evidence bags,related
6,6,"If Edward is a playwright, he'll bring his pen.",Edward has a pen.,E,possessive,his pen,related
7,7,"If William is a notary, he'll bring his notary...",William has a notary seal.,E,possessive,his notary seal,related


In [20]:
possessive_trigger = False
test_df = sample_test_df
# test_df = test_df4

In [21]:
test_example = [
    test_df['premise'], 
    test_df['hypothesis']
]

test_dataset = NewsDataset(
    data_list=test_example,
    tokenizer=tokenizer,
    max_length=256, 
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
)

In [22]:
integrated_grad = IntegratedGradient(
    model, 
    criterion, 
    tokenizer, 
    show_progress=False,
    # encoder="bert"
)
instances = integrated_grad.saliency_interpret(test_dataloader)

In [23]:
def reformat_output(instances):

    instances_word_gradients = []

    for instance in instances: 
        i = 0 # position counter 
        token_gradients = defaultdict(float) # dict to keep track of grad of each token     
        for token, grad in zip(instance['tokens'], instance['grad']):
            if token != '<unk>':
                token_gradients[(token,i)] += grad  # each token has its IG value added to the dict 
                i += 1
        
        token_gradients = dict(token_gradients)
        tokens = [tok[0] for tok in list(token_gradients.keys())]
        word_gradients = {}
    
        for i, token in enumerate(tokens):
            word = tokens[i]
            if word.startswith('Ġ'):
                word = word[1:]
            count = 1 
            word_gradient = token_gradients[(token,i)]
            if word in ['[SEP]','[CLS]']:
                 word_gradients[(word,i)] = word_gradient
                 continue # do not merge with next token
            while (i+1)<len(tokens) and tokens[i+1] not in string.punctuation and not tokens[i+1].startswith('Ġ') and not tokens[i+1] == '[SEP]':
                # part of same word, add gradients together and then average out 
                i += 1 # move to next word
                word += tokens[i] # concatente to restore word 
                word_gradient += token_gradients[(tokens[i],i)]
                count += 1
            # end of word 
            word_gradients[(word,i)] = word_gradient/count # averaging
        instances_word_gradients.append(word_gradients)
    
    return instances_word_gradients

In [24]:
def get_average_gradient(instances):
    # average gradient per token
    return [sum(instance['grad'])/len(instance['grad']) for instance in instances]

In [25]:
def compute_trigger_gradients(row):
    
    gradients = 0
    count = 0
    trigger_words = []

    # split phrases into individual words
    trigger_words = row['key_phrase'].split()

    # get gradient of all words in this example
    word_gradients = row['word_gradients']
    word_gradients_words = [key[0] for key in list(word_gradients.keys())] # extracting word portion of keys
    
    word_index_mapping = {}
    print(word_gradients.keys())
    
    for word, index in list(word_gradients.keys()):
        word_index_mapping.setdefault(word,index) # set the first value (premise value) as the index
    
    for word in trigger_words:
        if word in word_gradients_words:
            word_index = word_index_mapping[word]
            gradients += word_gradients[(word,word_index)]
            count += 1 
        elif word.lower() in word_gradients_words:
            word_index = word_index_mapping[word.lower()]
            gradients += word_gradients[(word.lower(),word_index)]
            count += 1 
        else:
            print(f"{row.name}: {word} not found!")

    # compute avg gradient of trigger tokens
    return gradients/count if count > 0 else 0

In [26]:
def get_predicted_label(instances):
    return [instance['label'] for instance in instances]

In [28]:
test_df['word_gradients'] = reformat_output(instances)
test_df.to_csv('part4b_type4_deberta_raw.csv')

In [ ]:
test_df['average_gradient'] = get_average_gradient(instances)
test_df['trigger_gradient'] = test_df.apply(compute_trigger_gradients,axis=1)
test_df['predicted_label'] = get_predicted_label(instances)
test_df['predicted_label'] = test_df['predicted_label'].map({0:'C',1:'N',2:'E'})
test_df['Trigger IG Ratio'] = test_df['trigger_gradient'] / test_df['average_gradient']
test_df

In [ ]:
# average Trigger IG Ratio
orig_data = test_df[test_df['type'] == 'related']['Trigger IG Ratio']
new_data = test_df[test_df['type'] != 'related']['Trigger IG Ratio']
orig_data.mean(), new_data.mean()

In [ ]:
test_df.to_csv('part4b_type4_deberta_results_new.csv')